# Налаштування середовища
У цьому розділі встановлюємо необхідні бібліотеки та автентифікуємося (за потреби) у Hugging Face. 

In [ ]:
!pip install --quiet diffusers transformers accelerate safetensors!pip install --quiet xformers  # За необхідності прискорити інференс
# За бажанням можна увійти в обліковий запис Hugging Face:
# from huggingface_hub import notebook_login
# notebook_login()

# Імпорт необхідних модулів

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler, EulerAncestralDiscreteScheduler,
DPMSolverMultistepScheduler, DDIMScheduler, StableDiffusionXLImg2ImgPipeline
from PIL import Image
import requests
import os


# Завантаження моделі
Завантажуємо модель з Hugging Face, а також видаляємо або збільшуємо обмеження токенізатора, щоб зняти ліміт символів.

In [ ]:
model_id = "Laxhar/noobai-XL-1.1"

# Створюємо пайплайн
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

# Встановлюємо довільно велике значення довжини токенів для обох токенізаторів
pipe.tokenizer.model_max_length = 1000000
pipe.tokenizer_2.model_max_length = 1000000

# За замовчуванням використовується EulerDiscreteScheduler;
# щоб легко перемикатись, створимо словник можливих scheduler:
schedulers = {
    "euler": EulerDiscreteScheduler.from_config(pipe.scheduler.config),
    "euler_ancestral": EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config),
    "dpm": DPMSolverMultistepScheduler.from_config(pipe.scheduler.config),
    "ddim": DDIMScheduler.from_config(pipe.scheduler.config)
}

def set_scheduler(pipeline, scheduler_name="euler"):
    if scheduler_name in schedulers:
        pipeline.scheduler = schedulers[scheduler_name]
    else:
        print(f"Невідомий scheduler: {scheduler_name}. Використовую euler за замовчуванням.")
        pipeline.scheduler = schedulers["euler"]

set_scheduler(pipe, "euler")  # За замовчуванням

# Функція для генерації зображень
Тут можна налаштовувати:
- Метод семплінгу (scheduler_name)
- Кількість кроків (num_inference_steps)
- Seed (generator)
- CFG scale (guidance_scale)
- Розмір вихідного зображення (width, height)
- Проста підтримка hires fix.

Для hires fix (наприклад 8x-NKMD-Superscale) – умовно показуємо, як можна збільшити роздільну здатність після генерації.
Зверніть увагу, що реальне масштабування 8x потребує додаткового upscaler-модуля (наприклад, ESRGAN або інші). Тут наведено приклад умовної реалізації.

In [ ]:
def generate_image(
        prompt,
        negative_prompt=None,
        scheduler_name="euler",
        num_inference_steps=30,
        seed=0,
        guidance_scale=7.5,
        width=512,
        height=512,
        hires_fix=False,
        hires_scale=2
    ):
    # Встановлюємо потрібний scheduler
    set_scheduler(pipe, scheduler_name)

    # Відтворюваність
    generator = torch.Generator("cuda").manual_seed(seed)

    # Генерація зображення
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=generator,
        width=width,
        height=height
    ).images[0]

    if hires_fix:
        # Умовна логіка: масштабувати зображення
        # Реалізація залежить від обраного інструменту. Нижче – проста PIL-версія.
        new_width = width * hires_scale
        new_height = height * hires_scale
        image = image.resize((new_width, new_height), resample=Image.LANCZOS)
        # Для 8x можна заданий hires_scale=8

    return image

# Приклад виклику:
# img = generate_image(
#     prompt="A cute cat, highly detailed",
#     negative_prompt="",
#     scheduler_name="euler",
#     num_inference_steps=25,
#     seed=123,
#     guidance_scale=7.5,
#     width=512,
#     height=512,
#     hires_fix=True,
#     hires_scale=2
# )
# img

# Використання
1. Встановіть або змініть між різними методами семплінгу: "euler", "euler_ancestral", "dpm", "ddim"
2. Налаштуйте кількість кроків (num_inference_steps)
3. Оберіть seed для відтворюваності
4. Задайте CFG scale (guidance_scale)
5. Виберіть роздільну здатність, а також увімкніть hires_fix, якщо потрібно
6. Зверніть увагу, що реальна "8x-NKMD-Superscale" вимагатиме іншого/додаткового спеціалізованого апскейлера.

Після запуску всіх ячейок ви можете викликати функцію `generate_image(...)` з потрібними параметрами.